### Imports

In [1]:
import sys
sys.path.insert(0, 'D:\Workspace\workspace_masterarbeit\PainLevelShiftDetection\Classes')

import warnings
warnings.filterwarnings('ignore')

from models import ClassificationModel, EmbeddingsModel, SiameseModel
from trainer import SiameseTrainerCombinationDataset, all_subjects_intense

import torch
import csv
from tqdm import tqdm
import os

### Set Hyperparameters

In [20]:
saving_path = "D:\Workspace\workspace_masterarbeit\PainLevelShiftDetection\TransferLearning\\results\model_test_results.csv"
model_dir_path = "D:\Workspace\workspace_masterarbeit\PainLevelShiftDetection\TransferLearning\pretrained_models\\"

# model LOOP

In [21]:
all_models = os.listdir(model_dir_path)
all_models = [model for model in all_models if model.endswith(".pth")]

In [ ]:
for model_path in tqdm(all_models):

    trainer_hyperparameter = {
        "path": "D:\Workspace\workspace_masterarbeit\PainLevelShiftDetection\FeatureGeneration\dataset_processed\INTENSE\\normalized_subjects_no_outliner.pkl",
        "subjects_train": all_subjects_intense[:1],
        "subjects_test": all_subjects_intense,
        "wandb": False,
        "log": True,
        "learning_rate": 0.001,
        "batch_size": 256,
        "batch_size_test": 256,
        "freeze_embed": False,
        "dataset_ignore_subject_train": True,
        "dataset_ignore_subject_test": False,
        "decision_function": 0,
        "filter": [],
        "weight_decay": 0,
        "lr_steps": 10,
        "adam": True,
        "number_steps": None,
        "number_steps_testing": None,
        "number_steps_histogramm": None
    }

    model_embedding_hyperparameter = {
            "sigmoid": False,
            "dropout": 0.35,
            "layers": [207, 512, 64]
    }

    model_classifier_hyperparameter = {
        "dropout": 0.35,
        "layers": [64, 32],
        "multiclass": False
    }

    #init all models
    classifier_model = ClassificationModel(model_classifier_hyperparameter)
    embedding_model = EmbeddingsModel(model_embedding_hyperparameter)
    siamese_model = SiameseModel(embedding_model, classifier_model, decision_function=trainer_hyperparameter["decision_function"])

    #load model
    _ = siamese_model.load_state_dict(torch.load(model_dir_path+model_path))

    #init trainer with model and hyperparameters
    trainer = SiameseTrainerCombinationDataset(trainer_hyperparameter, siamese_model, device="cuda:0")

    #test model on intense dataset
    test_results = trainer.test()
    cm = test_results["cm"]
    acc = test_results["acc"]
    results = trainer.calculate_f_scores(cm)
    recall = results["recall"]
    precision = results["recall"]
    f1 = results["recall"]

    #save results
    with open(saving_path, mode='a') as result_file:
            results_writer = csv.writer(result_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
            results_writer.writerow([model_path[:-4], acc*100, recall, precision, f1])